# Create Patch Dataset
This notebook is the source for downloading Sentinel data for the 2D patch classifier

## Inputs
Sites are generated from geojson inputs. The positive TPA sites are defined by polygons, and the negative sites are defined by points. 
New negative sites can be added by following the example of the bootstrap dataset

## Outputs:

### Raw Data:
The output is a dictionary with a structure `[date][site_name][band][band_img]`

### Patches
The output is multispectral patches. `[num_patches][height][width][bands]`

In [10]:
import json
import os
import pickle

import ee
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import sys
sys.path.append('../')
from scripts.get_s2_data_ee import get_history, get_history_polygon, get_pixel_vectors

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Configuration:
# Set directory where training site json files are located and files are saved
# Set rect width for all patches that are not TPA sites
DATA_DIR = '../data/training_sites'
OUTPUT_DIR = '../data/sentinel'
RECT_WIDTH = 0.0075
    
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [4]:
# Sentinel 2 band descriptions
band_descriptions = {
    'B1': 'Aerosols, 442nm',
    'B2': 'Blue, 492nm',
    'B3': 'Green, 559nm',
    'B4': 'Red, 665nm',
    'B5': 'Red Edge 1, 704nm',
    'B6': 'Red Edge 2, 739nm',
    'B7': 'Red Edge 3, 779nm',
    'B8': 'NIR, 833nm',
    'B8A': 'Red Edge 4, 864nm',
    'B9': 'Water Vapor, 943nm',
    'B11': 'SWIR 1, 1610nm',
    'B12': 'SWIR 2, 2186nm'
}

In [5]:
def load_points(file_name):
    """Load points saved as a GeoJSON and return a dictionary"""
    with open(os.path.join(DATA_DIR, file_name)) as f:
        sites = json.load(f)
    f.close()

    site_table = pd.DataFrame({
        'name': [file_name.split('_')[0] + '_' + str(index) for index in range(len(sites['features']))],
        'lon': [site['geometry']['coordinates'][0] for site in sites['features']],
        'lat': [site['geometry']['coordinates'][1] for site in sites['features']],
        'coords': [site['geometry']['coordinates'][0:2] for site in sites['features']],
    })
    
    return site_table


def sample_adjacent(tpa_sites, offset, direction='east'):
    """
    Outputs a data frame of sampling locations based on a distance
    and direction from each TPA site.
    This can be used for adjacent site sampling, or to create "random" negative sites if the
    offset distance is set further away from the TPA location.
    Returns a data frame
    """
    if  'east' in direction.lower():
        adjacent_sites = pd.DataFrame({
            'name': [f"{name}_{direction.lower()}_{offset}" for name in tpa_sites['name']],
            'lon': [lon + offset for lon in tpa_sites['lon']],
            'lat': [lat for lat in tpa_sites['lat']],
            'coords': [[lon + offset, lat] for lon, lat in zip(tpa_sites['lon'], tpa_sites['lat'])]
        })
        
    if  'west' in direction.lower():
        adjacent_sites = pd.DataFrame({
            'name': [f"{name}_{direction.lower()}_{offset}" for name in tpa_sites['name']],
            'lon': [lon - offset for lon in tpa_sites['lon']],
            'lat': [lat for lat in tpa_sites['lat']],
            'coords': [[lon + offset, lat] for lon, lat in zip(tpa_sites['lon'], tpa_sites['lat'])]
        })
    
    if  'north' in direction.lower():
        adjacent_sites = pd.DataFrame({
            'name': [f"{name}_{direction.lower()}_{offset}" for name in tpa_sites['name']],
            'lon': [lon for lon in tpa_sites['lon']],
            'lat': [lat + offset for lat in tpa_sites['lat']],
            'coords': [[lon + offset, lat] for lon, lat in zip(tpa_sites['lon'], tpa_sites['lat'])]
        })
    
    if  'south' in direction.lower():
        adjacent_sites = pd.DataFrame({
            'name': [f"{name}_{direction.lower()}_{offset}" for name in tpa_sites['name']],
            'lon': [lon for lon in tpa_sites['lon']],
            'lat': [lat - offset for lat in tpa_sites['lat']],
            'coords': [[lon + offset, lat] for lon, lat in zip(tpa_sites['lon'], tpa_sites['lat'])]
        })
    
    return adjacent_sites

## Load Sampling Locations

### Positive Sites

In [6]:
# Positive site coordinates

with open('../data/detection_summary_v12.geojson', 'r') as f:
    positive_sites = json.load(f)['features']
positive_coords = [site['geometry']['coordinates'] for site in positive_sites]
positive_names = [site['properties']['name'] for site in positive_sites]
print(len(positive_sites), 'positive sites loaded')

17 positive sites loaded


### Negative Sites

In [6]:
# Create a list of dataframes for default negative site sampling
# Some lists are specifically chosen, and some are generated 
# automatically based on the locations of TPA sites
negative_site_list = [load_points('city_points_30.geojson'), 
                      load_points('bare_earth_points.geojson'),
                      sample_adjacent(tpa_sites, 0.01, 'north')]

for site in negative_site_list:
    display(site.head())

,name,lon,lat,coords
0,city_0,114.619837,-8.361932,"[114.6198374623975, -8.361931821454325]"
1,city_1,115.218992,-8.682543,"[115.2189915773064, -8.682542635447703]"
2,city_2,115.152099,-8.803352,"[115.1520991337562, -8.803351890677076]"
3,city_3,115.448223,-8.676354,"[115.4482234100242, -8.676354239123828]"
4,city_4,115.552125,-8.674258,"[115.5521252514949, -8.674258048038155]"


,name,lon,lat,coords
0,bare_0,115.571182,-8.409224,"[115.5711823425384, -8.409223580393347]"
1,bare_1,115.383874,-8.562584,"[115.3838741922983, -8.562583709771332]"
2,bare_2,115.554872,-8.389197,"[115.5548717252287, -8.389196879431212]"
3,bare_3,115.233932,-8.670693,"[115.2339323116851, -8.670692512141244]"
4,bare_4,115.172205,-8.680163,"[115.1722049239458, -8.68016332592915]"


,name,lon,lat,coords
0,TPA Jungut Batu_north_0.01,115.459414,-8.660958,"[115.46941439485306, -8.670958330781342]"
1,TPA Biaung_north_0.01,115.498017,-8.669930,"[115.50801683267277, -8.679930042100876]"
2,TPA Sente_north_0.01,115.454460,-8.520372,"[115.46446033358268, -8.530371792768301]"
3,TPA Regional Bangli_north_0.01,115.367927,-8.343542,"[115.37792701853951, -8.353541681392851]"
4,TPA Peh_north_0.01,114.583295,-8.317938,"[114.59329467897307, -8.327937523143966]"


In [7]:
# New data should likely be added to the bootstrap negatives file
# This will be a part of the negative dataset, but is separate for an example of 
# adding new data to an existing dataset
bootstrap_negatives = load_points('bootstrap_negatives.geojson')
bootstrap_negatives.head()

,name,lon,lat,coords
0,bootstrap_0,115.112336,-8.845362,"[115.112336, -8.845362]"
1,bootstrap_1,115.119432,-8.798492,"[115.119432, -8.798492]"
2,bootstrap_2,115.131650,-8.789526,"[115.13165, -8.789526]"
3,bootstrap_3,115.186896,-8.845007,"[115.186896, -8.845007]"
4,bootstrap_4,115.187076,-8.841368,"[115.187076, -8.841368]"


In [8]:
negative_data = pd.concat([negative_site_list[0], 
                           negative_site_list[1], 
                           negative_site_list[2], 
                           bootstrap_negatives]).reset_index(drop=True)

negative_data.to_csv(os.path.join(DATA_DIR, 'negative_sites.csv'))
print(len(negative_data), 'negative sites')
negative_data.head()

104 negative sites


,name,lon,lat,coords
0,city_0,114.619837,-8.361932,"[114.6198374623975, -8.361931821454325]"
1,city_1,115.218992,-8.682543,"[115.2189915773064, -8.682542635447703]"
2,city_2,115.152099,-8.803352,"[115.1520991337562, -8.803351890677076]"
3,city_3,115.448223,-8.676354,"[115.4482234100242, -8.676354239123828]"
4,city_4,115.552125,-8.674258,"[115.5521252514949, -8.674258048038155]"


## Download Data

### Negative Sites

In [9]:
# Create a list of patch histories
# Each patch history is a dictionary with the format:
# patch_history[date][site_name][band][band_img]
# This function takes a while to run as it is extracting data from GEE
num_negative_months = 12
negative_start_date = '2019-01-01'
negative_patch_histories = get_history(negative_data['coords'], 
                                       negative_data['name'], 
                                       RECT_WIDTH,
                                       num_months=num_negative_months,
                                       start_date=negative_start_date,
                                       cloud_mask=True)

  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [08:06<1:29:10, 486.40s/it]

 17%|█▋        | 2/12 [16:27<1:21:47, 490.77s/it]

 25%|██▌       | 3/12 [23:41<1:11:04, 473.88s/it]

 33%|███▎      | 4/12 [37:49<1:18:07, 585.95s/it]

...

 67%|██████▋   | 8/12 [1:49:01<1:04:27, 966.79s/it]

 75%|███████▌  | 9/12 [2:05:12<48:24, 968.18s/it]  

 83%|████████▎ | 10/12 [2:19:35<31:13, 936.61s/it]

 92%|█████████▏| 11/12 [2:33:32<15:06, 906.62s/it]

100%|██████████| 12/12 [2:43:37<00:00, 818.10s/it]


In [10]:
# Save negative patch history
with open(os.path.join(OUTPUT_DIR, f"negative_raw_toa_{RECT_WIDTH}_patch_{num_negative_months}_{negative_start_date}.pkl"),"wb") as f:
    pickle.dump(negative_patch_histories, f)

### Positive Sites

In [11]:
# Get patch histories for TPA sites
num_positive_months = 36
positive_start_date = '2018-01-01'
positive_history = get_history(positive_coords, 
                               positive_names,  
                               RECT_WIDTH,
                               start_date=positive_start_date,
                               num_months=num_positive_months
                              )

  0%|          | 0/36 [00:00<?, ?it/s]

Processing TPA Jungut Batu
Processing TPA Biaung
Processing TPA Sente
Processing TPA Regional Bangli
Processing TPA Peh
Processing TPA Temesi
Processing TPA Bengkala
Processing TPA Bebandem
Processing TPA Mandung
Processing TPA Regional Suwung


  3%|▎         | 1/36 [00:34<19:54, 34.14s/it]

Processing TPA Jungut Batu
Processing TPA Biaung
Processing TPA Sente
Processing TPA Regional Bangli
Processing TPA Peh
Processing TPA Temesi
Processing TPA Bengkala
Processing TPA Bebandem
Processing TPA Mandung
Processing TPA Regional Suwung


  6%|▌         | 2/36 [01:21<21:33, 38.04s/it]

...

 94%|█████████▍| 34/36 [30:33<01:46, 53.03s/it]

Processing TPA Jungut Batu
Processing TPA Biaung
Processing TPA Sente
Processing TPA Regional Bangli
Processing TPA Peh
Processing TPA Temesi
Processing TPA Bengkala
Processing TPA Bebandem
Processing TPA Mandung
Processing TPA Regional Suwung


 97%|█████████▋| 35/36 [31:28<00:53, 53.66s/it]

Processing TPA Jungut Batu
Processing TPA Biaung
Processing TPA Sente
Processing TPA Regional Bangli
Processing TPA Peh
Processing TPA Temesi
Processing TPA Bengkala
Processing TPA Bebandem
Processing TPA Mandung
Processing TPA Regional Suwung


100%|██████████| 36/36 [31:59<00:00, 53.31s/it]


In [12]:
# Save positive patch history
with open(os.path.join(OUTPUT_DIR, f"positive_raw_toa_{RECT_WIDTH}_patch_{num_positive_months}_{positive_start_date}.pkl"),"wb") as f:
    pickle.dump(tpa_patch_histories, f)

## Create Spatial Patches

In [ ]:
def create_img_stack(patch_history):
    img_stack = []
    for date in patch_history:
        for site in patch_history[date]:
            spectral_stack = []
            band_shapes = [np.shape(patch_history[date][site][band]) for band in band_descriptions]
            if np.array(band_shapes).all() > 0:
                for band in band_descriptions:
                    spectral_stack.append(patch_history[date][site][band])
                if np.min(spectral_stack) > 0:
                    img_stack.append(np.rollaxis(np.array(spectral_stack), 0, 3))
    return img_stack

In [ ]:
positive_patches = create_img_stack(tpa_patch_histories)
print(len(positive_patches), 'positive images extracted')

negative_patches = create_img_stack(negative_patch_histories)
print(len(negative_patches), 'negative images extracted')

In [ ]:
# Save positive patch history
with open(os.path.join(OUTPUT_DIR, f"positive_patches_toa_{num_positive_months}_{positive_start_date}.pkl"),"wb") as f:
    pickle.dump(positive_patches, f)
    
# Save negative patch history
with open(os.path.join(OUTPUT_DIR, f"negative_patches_toa_{num_positive_months}_{positive_start_date}.pkl"),"wb") as f:
    pickle.dump(negative_patches, f)

## Debug and Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.lines import Line2D

In [ ]:
def animate_patch_history(data, name):
    """
    Used for visualization and debugging. Takes a history dictionary and outputs a video
    for each timestep at each site in the history.
    """
    fig, ax = plt.subplots(dpi=100, facecolor=(1,1,1))
    ax.set_axis_off()
    images = []
    init_date = list(data.keys())[0]
    for site_name in data[init_date]:
        for date in data.keys():
            ax.set_title(name)
            hyperpatch = data[date][site_name]
            rgb = np.stack((hyperpatch['B4'], hyperpatch['B3'], hyperpatch['B2']), axis=-1)
            if len(rgb) > 0:
                im = plt.imshow(rgb / 2000, animated=True)
                images.append([im])
    fig.tight_layout()
    print(site_name.split('_')[1:])
    ani = animation.ArtistAnimation(fig, images, interval=60, blit=True, repeat_delay=500)
    ani.save(os.path.join('figures', 'videos', name + '.mp4'))

In [ ]:
animate_patch_history(tpa_patch_histories, 'TPA - TOA')
animate_patch_history(patch_histories[0], 'City - TOA')